## Simple LangGraph Chatbot

In [2]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from typing_extensions import TypedDict

MODEL = "llama2"
MODEL_PROVIDER = "ollama"

llm = init_chat_model(
    model=f"{MODEL_PROVIDER}:{MODEL}"
)


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)


def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

user_input = input("> ")
state = graph.invoke({"messages": [{"role": "user", "content": user_input}]})

print(state["messages"])
print(state["messages"][-1].content)

[HumanMessage(content='what is the square of 13?', additional_kwargs={}, response_metadata={}, id='8ab90530-1da3-4358-bf7b-5a72a6f0b087'), AIMessage(content='The square of 13 is 169.', additional_kwargs={}, response_metadata={'model': 'llama2', 'created_at': '2025-11-15T08:38:26.537566Z', 'done': True, 'done_reason': 'stop', 'total_duration': 724313125, 'load_duration': 40833459, 'prompt_eval_count': 29, 'prompt_eval_duration': 275100708, 'eval_count': 13, 'eval_duration': 384804248, 'model_name': 'llama2', 'model_provider': 'ollama'}, id='lc_run--1f06e819-6392-49e1-9fcc-b4ff8aa6c6a8-0', usage_metadata={'input_tokens': 29, 'output_tokens': 13, 'total_tokens': 42})]
The square of 13 is 169.


## Complex LangGraph Chatbot

In [ ]:
from typing import Annotated, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import init_chat_model
from pydantic import BaseModel, Field
from typing_extensions import TypedDict

MODEL = "llama2"
MODEL_PROVIDER = "ollama"

llm = init_chat_model(
    model=f"{MODEL_PROVIDER}:{MODEL}"
)


class MessageClassifier(BaseModel):
    message_type: Literal["emotional", "logical"] = Field(
        ...,
        description="Classify if the message requires an emotional (therpaist) or logical response."
    )


class State(TypedDict):
    messages: Annotated[list, add_messages]
    message_type: str | None


def classify_message(state: State):
    last_message = state["messages"][-1]
    classifier_llm = llm.with_structured_output(MessageClassifier)

    result = classifier_llm.invoke([
        {
            "role": "system",
            "content": """Classify the user message as either:
            - 'emotional': if it asks for emotional support, therapy, deals with feelings, or personal problems
            - 'logical': if it asks for facts, information, logical analysis, or practical solutions""",
        },
        {
            "role": "user",
            "content": last_message.content,
        }
    ])
    return {"message_type": result.message_type}


def router(state: State):
    message_type = state.get("message_type", "logical")
    if message_type == "emotional":
        return {"next": "therapist"}
    return {"next": "logical"}


def therapist_agent(state: State):
    last_message = state["messages"][-1]

    messages = [
        {
            "role": "system",
            "content": """You are a compassionate therapist. Focus on the emotional aspects of the user's message.
            Show empathy, validate their feelings, and help them process their emotions.
            Ask thoughtful questions to help them explore their feelings more deeply.
            Avoid giving logical solutions unless explicitly asked."""
        },
        {
            "role": "user",
            "content": last_message.content
        },
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": reply.content}]}


def logical_agent(state: State):
    last_message = state["messages"][-1]

    messages = [
        {
            "role": "system",
            "content": """You are a purely logical assistant. Focus only on facts and information.
            Provide clear, concise answers based on logic and evidence.
            Do not address emotions or provide emotional support.
            Be direct and straightforward in your responses."""
        },
        {
            "role": "user",
            "content": last_message.content
        },
    ]
    reply = llm.invoke(messages)
    return {"messages": [{"role": "assistant", "content": reply.content}]}


graph_builder = StateGraph(State)

graph_builder.add_node("classifier", classify_message)
graph_builder.add_node("router", router)
graph_builder.add_node("therapist", therapist_agent)
graph_builder.add_node("logical", logical_agent)

graph_builder.add_edge(START, "classifier")
graph_builder.add_edge("classifier", "router")

graph_builder.add_conditional_edges(
    "router",
    lambda state: state.get("next"),
    {"therapist": "therapist", "logical": "logical"},
)

graph_builder.add_edge("therapist", END)
graph_builder.add_edge("logical", END)

graph = graph_builder.compile()


def run_chatbot():
    state = {"messages": [], "message_type": None}

    while True:
        user_input = input("> ")
        if user_input == "exit":
            print("Bye")
            break

        state["messages"] = state.get("messages", []) + [
            {"role": "user", "content": user_input}
        ]

        state = graph.invoke(state)

        if state.get("messages") and len(state.get("messages")) > 0:
            last_message = state["messages"][-1]
            print(f"Assistant: {last_message.content}")


if __name__ == "__main__":
    run_chatbot()
    